<h4>## ETAPA 1.0##</h4>

In [1]:
# Importar librerías principales
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

<h4>## ETAPA 1.1 + 1.2 ##</h4>

In [9]:
# Creacion de diccionario para resolver una incidencia sobre la columna "fecha"
meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

# Cargar archivo CSV con parámetros regionales
ruta_archivo = 'raw/avance_rg.csv'
df = pd.read_csv(ruta_archivo, sep=';', decimal=',', encoding='utf-8')
print('Archivo cargado correctamente✔️')

# Eliminar nombre del día y convertir texto a formato dd/mm/yyyy
df['fecha'] = df['fecha'].str.replace(r'^\w+,\s*', '', regex=True)

# Reemplazar nombre de mes por número
for mes, num in meses.items():
    df['fecha'] = df['fecha'].str.replace(f' de {mes} de ', f'/{num}/', regex=False)

# Ahora convierte la columna 'fecha' a datetime
df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y', errors='coerce')
print('Fecha convertida con éxito al formato dd/mm/aaaa✔️')

# Verificar primeras filas para control visual
print(df.head())

Archivo cargado correctamente✔️
Fecha convertida con éxito al formato dd/mm/aaaa✔️
     proveedor                                           producto      fecha  \
0  Maxiconsumo  Toallitas Humedas Cuidado Diario Baby Estrella... 2024-05-07   
1  Maxiconsumo  Toallitas Humedas Cuidado Diario Baby Estrella... 2024-05-08   
2  Maxiconsumo  Toallitas Humedas Cuidado Diario Baby Estrella... 2024-05-09   
3  Maxiconsumo  Toallitas Humedas Cuidado Diario Baby Estrella... 2024-05-10   
4  Maxiconsumo  Toallitas Humedas Cuidado Diario Baby Estrella... 2024-05-12   

   precio  
0  2099.9  
1  2099.9  
2  2099.9  
3  2099.9  
4  2099.9  


<h4>## ETAPA 1.3##</h4>

In [11]:
# Reemplazar precios iguales a 0 usando interpolación dentro de una ventana móvil de 10 días
df['precio_validado'] = df['precio']  # Crea una nueva columna sin alterar el original

# Convertir a datetime por si fuera necesario para asegurar el orden
df = df.sort_values(by=['producto', 'proveedor', 'fecha'])

# Recorremos cada grupo de producto-proveedor
for (prod, prov), grupo in df.groupby(['producto', 'proveedor']):
    # Interpolación por fecha solo en valores no nulos ni cero
    grupo = grupo.copy()
    grupo.loc[grupo['precio_validado'] == 0, 'precio_validado'] = np.nan
    grupo['precio_validado'] = grupo.set_index('fecha')['precio_validado'].interpolate(method='time', limit=10, limit_direction='both').values
    df.loc[grupo.index, 'precio_validado'] = grupo['precio_validado']
    
#Print para verificar que se hayan corregido los valores
print('Precios igual a 0 corregidos usando ventana temporal de 10 días✔️')

# Mostrar registros originales con precio 0 y su valor corregido
print(df[df['precio'] == 0][['producto', 'proveedor', 'fecha', 'precio', 'precio_validado']].head())

Precios igual a 0 corregidos usando ventana temporal de 10 días✔️
                      producto   proveedor      fecha  precio  precio_validado
521215   Conejo Rocklets 55 Gr   Carrefour 2025-04-07     0.0           5990.0
575631   Conejo Rocklets 55 Gr        Dasa 2025-04-07     0.0              NaN
574183   Conejo Rocklets 55 Gr      Diarco 2025-04-07     0.0              NaN
304169   Conejo Rocklets 55 Gr  La Anonima 2025-04-07     0.0           5300.0
634289   Conejo Rocklets 55 Gr  Limp Hogar 2025-04-07     0.0              NaN


<h4>## ETAPA 1.4 ##</h4>

In [14]:
# Identificar precios muy bajos en la columna validada
cond_valores_pequenos = (df['precio_validado'] < 10) & (df['precio_validado'] > 0)
df.loc[cond_valores_pequenos, 'precio_validado'] = df.loc[cond_valores_pequenos, 'precio_validado'] * 100

#Print para verificar que se aplicaron los cambios
print('Precios que aparecian como decimales corregidos automáticamente (x100)✔️')

# Mostrar registros corregidos por haber sido valores extremadamente pequeños
print(df[(df['precio'] < 10) & (df['precio_validado'] >= 10)][['producto', 'proveedor', 'fecha', 'precio', 'precio_validado']].head())

Precios que aparecian como decimales corregidos automáticamente (x100)✔️
                                                 producto   proveedor  \
521215                              Conejo Rocklets 55 Gr   Carrefour   
304169                              Conejo Rocklets 55 Gr  La Anonima   
406534   Don Miguel Budín Don Miguel Marmolado Bolsa 1...   Carrefour   
406535     Fiesta Ananá Fizz Sin Alcohol Fiesta Pet 1 Lt.   Carrefour   
406786     Fiesta Ananá Fizz Sin Alcohol Fiesta Pet 1 Lt.   Carrefour   

            fecha  precio  precio_validado  
521215 2025-04-07     0.0           5990.0  
304169 2025-04-07     0.0           5300.0  
406534 2024-11-22     0.0            990.0  
406535 2024-11-14     0.0           3000.0  
406786 2024-12-09     0.0           3000.0  


<h4>## ETAPA 1.5 ##</h4>

In [20]:
# Importamos el modelo de detección de anomalías
from sklearn.ensemble import IsolationForest

# Inicializamos la columna 'outlier' en 0 (sin detectar)
df['outlier'] = 0

# Definimos el modelo de detección
modelo = IsolationForest(contamination=0.01, random_state=42)

# Agrupamos el dataset por producto y proveedor
grupos = list(df.groupby(['producto', 'proveedor']))
total = len(grupos)
paso = max(1, total // 20)  # Avance por porcentaje (~5%)

# Recorremos cada grupo y aplicamos el modelo si hay datos suficientes
for i, ((prod, prov), grupo) in enumerate(grupos):
    grupo = grupo.copy()
    grupo_validos = grupo[['precio_validado']].dropna()

    if len(grupo_validos) > 10:
        pred = modelo.fit_predict(grupo_validos)
        indices_detectados = grupo_validos.index[pred == -1]
        df.loc[indices_detectados, 'outlier'] = 1

    if i % paso == 0 or i == total - 1:
        porcentaje = round((i+1) / total * 100, 1)
        print(f'Procesando grupo {i+1}/{total} → {porcentaje}% 🔄')

#Verificación inicial de detección completada.
print('Detección completa con IsolationForest aplicada a todo el dataset ✔️')

# Verificación visual final
print("Muestra de registros detectados como posibles precios inflados 🔍")
display(df[df['outlier'] == 1].head(10))

Procesando grupo 1/10537 → 0.0% 🔄
Procesando grupo 527/10537 → 5.0% 🔄
Procesando grupo 1053/10537 → 10.0% 🔄
Procesando grupo 1579/10537 → 15.0% 🔄
Procesando grupo 2105/10537 → 20.0% 🔄
Procesando grupo 2631/10537 → 25.0% 🔄
Procesando grupo 3157/10537 → 30.0% 🔄
Procesando grupo 3683/10537 → 35.0% 🔄
Procesando grupo 4209/10537 → 39.9% 🔄
Procesando grupo 4735/10537 → 44.9% 🔄
Procesando grupo 5261/10537 → 49.9% 🔄
Procesando grupo 5787/10537 → 54.9% 🔄
Procesando grupo 6313/10537 → 59.9% 🔄
Procesando grupo 6839/10537 → 64.9% 🔄
Procesando grupo 7365/10537 → 69.9% 🔄
Procesando grupo 7891/10537 → 74.9% 🔄
Procesando grupo 8417/10537 → 79.9% 🔄
Procesando grupo 8943/10537 → 84.9% 🔄
Procesando grupo 9469/10537 → 89.9% 🔄
Procesando grupo 9995/10537 → 94.9% 🔄
Procesando grupo 10521/10537 → 99.8% 🔄
Procesando grupo 10537/10537 → 100.0% 🔄
Detección completa con IsolationForest aplicada a todo el dataset ✔️
Muestra de registros detectados como posibles precios inflados 🔍


,proveedor,producto,fecha,precio,precio_validado,outlier
573010,Diarco,Fresita Espumante Fresita 750 Cc.,2024-12-26,7400.0,7400.0,1
223798,La Anonima,Fresita Espumante Fresita 750 Cc.,2025-03-13,82000.0,82000.0,1
248958,La Anonima,Gaona Budin Gaona Marmolado 180 G.,2024-12-27,1462.5,1462.5,1
223800,La Anonima,"Gaseosa Cola Coca Cola Sabor Liviano 2,25 Lts",2025-03-13,39500.0,39500.0,1
499137,Carrefour,Repuesto Escolar Carrefour N3 Cuadriculado 19...,2025-02-17,29990.0,29990.0,1
438731,Carrefour,Aceite De Girasol Carrefour Alto Omega Pet 900...,2024-09-28,1150.0,1150.0,1
465327,Carrefour,Aceite De Girasol Carrefour Alto Omega Pet 900...,2025-01-15,2089.0,2089.0,1
571880,Diarco,Aceite De Girasol Cañuelas 1.5 Lt.,2025-02-17,1881.0,1881.0,1
572686,Diarco,Aceite De Girasol Cañuelas 1.5 Lt.,2025-04-04,3476.0,3476.0,1
237900,La Anonima,Aceite De Girasol Cañuelas 1.5 Lt.,2024-08-14,1785.0,1785.0,1


<h4>## ETAPA 1.6 ##</h4>

In [23]:
from datetime import timedelta

# Creamos una nueva columna para reflejar la corrección final
df['precio_final'] = df['precio_validado']

# Filtramos solo los registros que fueron detectados como outliers
outliers_df = df[df['outlier'] == 1]

# Parámetros para la barra de progreso
total_outliers = len(outliers_df)
porcentaje_print = 5
proximo_print = porcentaje_print

# Recorremos cada outlier y aplicamos una corrección basada en ventana temporal
for idx, (i, fila) in enumerate(outliers_df.iterrows(), start=1):
    producto = fila['producto']
    proveedor = fila['proveedor']
    fecha = fila['fecha']
    
    # Definimos la ventana de 10 días
    fecha_min = fecha - timedelta(days=10)
    fecha_max = fecha + timedelta(days=10)
    
    # Buscamos precios válidos y normales en esa ventana temporal
    ventana = df[
        (df['producto'] == producto) &
        (df['proveedor'] == proveedor) &
        (df['fecha'] >= fecha_min) &
        (df['fecha'] <= fecha_max) &
        (df['outlier'] == 0) &
        (df['precio_validado'].notna())
    ]
    
    # Si hay al menos dos valores válidos, calculamos el promedio y corregimos
    if len(ventana) >= 2:
        promedio = ventana['precio_validado'].mean()
        df.at[i, 'precio_final'] = round(promedio, 2)
    
    # Progreso visual cada 5%
    avance = (idx / total_outliers) * 100
    if avance >= proximo_print:
        print(f"Progreso: {int(avance)}% completado ✔️")
        proximo_print += porcentaje_print

# Confirmamos finalización del proceso
print('Corrección automática de outliers completada ✔️')

# Verificación visual: mostramos outliers que fueron corregidos
print('Muestra de outliers corregidos automáticamente 🔍')
display(df[(df['outlier'] == 1) & (df['precio_final'] != df['precio_validado'])].head(10))

# Verificación visual final: Comparativa completa entre los tres estados del dato
print("Comparación entre precio original, validado y final 🔍")
display(
    df[(df['precio'] != df['precio_final'])][
        ['producto', 'proveedor', 'fecha', 'precio', 'precio_validado', 'precio_final']
    ].head(10))

Corrección automática de outliers completada ✔️
Muestra de outliers corregidos automáticamente 🔍


,proveedor,producto,fecha,precio,precio_validado,outlier,precio_final
573010,Diarco,Fresita Espumante Fresita 750 Cc.,2024-12-26,7400.0,7400.0,1,6734.85
223798,La Anonima,Fresita Espumante Fresita 750 Cc.,2025-03-13,82000.0,82000.0,1,8200.00
248958,La Anonima,Gaona Budin Gaona Marmolado 180 G.,2024-12-27,1462.5,1462.5,1,1828.12
223800,La Anonima,"Gaseosa Cola Coca Cola Sabor Liviano 2,25 Lts",2025-03-13,39500.0,39500.0,1,3950.00
499137,Carrefour,Repuesto Escolar Carrefour N3 Cuadriculado 19...,2025-02-17,29990.0,29990.0,1,12590.00
438731,Carrefour,Aceite De Girasol Carrefour Alto Omega Pet 900...,2024-09-28,1150.0,1150.0,1,1550.00
571880,Diarco,Aceite De Girasol Cañuelas 1.5 Lt.,2025-02-17,1881.0,1881.0,1,2756.50
572686,Diarco,Aceite De Girasol Cañuelas 1.5 Lt.,2025-04-04,3476.0,3476.0,1,3671.00
237900,La Anonima,Aceite De Girasol Cañuelas 1.5 Lt.,2024-08-14,1785.0,1785.0,1,2867.65
350346,La Anonima,Aceite De Girasol Cañuelas 1.5 Lt.,2024-12-08,2400.0,2400.0,1,3700.00


Comparación entre precio original, validado y final 🔍


,producto,proveedor,fecha,precio,precio_validado,precio_final
521215,Conejo Rocklets 55 Gr,Carrefour,2025-04-07,0.0,5990.0,5990.0
575631,Conejo Rocklets 55 Gr,Dasa,2025-04-07,0.0,NaN,NaN
574183,Conejo Rocklets 55 Gr,Diarco,2025-04-07,0.0,NaN,NaN
304169,Conejo Rocklets 55 Gr,La Anonima,2025-04-07,0.0,5300.0,5300.0
634289,Conejo Rocklets 55 Gr,Limp Hogar,2025-04-07,0.0,NaN,NaN
79354,Conejo Rocklets 55 Gr,Maxiconsumo,2025-04-07,0.0,NaN,NaN
667188,Conejo Rocklets 55 Gr,Mercado Concentrador De La Costa,2025-04-07,0.0,NaN,NaN
406534,Don Miguel Budín Don Miguel Marmolado Bolsa 1...,Carrefour,2024-11-22,0.0,990.0,990.0
575323,Don Miguel Budín Don Miguel Marmolado Bolsa 1...,Dasa,2024-11-22,0.0,NaN,NaN
573863,Don Miguel Budín Don Miguel Marmolado Bolsa 1...,Diarco,2024-11-22,0.0,NaN,NaN


<h4>## ETAPA 1.7 ##</h4>

In [25]:
# Creamos una nueva columna para aplicar la corrección decimal final
df['precio_final_corregido'] = df['precio_final']

# Aplicamos una corrección solo a valores numéricos, positivos y menores a 5
mascara = (
    df['precio_final_corregido'].notna() &
    (df['precio_final_corregido'] > 0) &
    (df['precio_final_corregido'] < 5)
)

# Multiplicamos por 100 a los valores detectados como posibles errores de decimalización
df.loc[mascara, 'precio_final_corregido'] = df.loc[mascara, 'precio_final_corregido'] * 100

# Confirmamos por consola que se aplicó la transformación
print("Corrección residual de decimales aplicada ✔️")

# Verificación visual: mostramos registros que fueron efectivamente modificados
print("🔍 Muestra de valores modificados en esta etapa:")
display(df[mascara][[
    'producto', 'proveedor', 'fecha', 'precio', 'precio_final', 'precio_final_corregido'
]].head(10))

Corrección residual de decimales aplicada ✔️
🔍 Muestra de valores modificados en esta etapa:


,producto,proveedor,fecha,precio,precio_final,precio_final_corregido


<h4>## Final del dia, se espera validacion ##</h4>

In [26]:
df.to_csv('processed/avance_rg_pandas.csv', sep=';', decimal=',', encoding='utf-8', index=False)
print("📦 Archivo de validación guardado correctamente ✔️")

📦 Archivo de validación guardado correctamente ✔️
